In [ ]:
# Importer les bibliothèques nécessaires
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates # Pour le formatage des dates
import ipywidgets as widgets # Pour les widgets interactifs
from ipywidgets import interact, interactive, fixed, HBox, VBox # Outils supplémentaires
from IPython.display import display, clear_output # Pour gérer l'affichage des widgets
import datetime # Pour les widgets de date

# --- Configuration Globale ---
try:
    NOTEBOOK_DIR = Path(__file__).resolve().parent
except NameError:
    NOTEBOOK_DIR = Path.cwd()
# Adaptez ce chemin si nécessaire :
# Si votre notebook est dans "S1/notebooks/" et data dans "data/" à la racine du projet
# DATA_DIR = NOTEBOOK_DIR.parent.parent / "data"
# Si votre notebook est dans "S1/" et data dans "data/"
DATA_DIR = NOTEBOOK_DIR.parent / "data"
# Si votre notebook est à la racine du projet et data dans "data/"
# DATA_DIR = NOTEBOOK_DIR / "data"

print(f"Répertoire de données configuré : {DATA_DIR}")
if not DATA_DIR.exists():
    print(f"⚠️  ATTENTION: Le répertoire de données {DATA_DIR} n'a pas été trouvé.")


# --- Styles Matplotlib ---
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({
    'figure.figsize': (16, 8),
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.family': 'sans-serif',
    'axes.labelsize': 12,
    'axes.titlesize': 16,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 12,
})

# --- Scan des fichiers et préparation des options des widgets ---
available_files = sorted(list(DATA_DIR.glob("*.parquet"))) # Trié pour un ordre cohérent
parsed_files_info = {} # Dictionnaire pour stocker les infos des fichiers

for f_path in available_files:
    parts = f_path.stem.split('_')
    if len(parts) >= 3 and parts[-1].endswith('d') and 'ta' not in parts:
        symbol = parts[0].upper()
        interval = parts[1]
        days_str = parts[2].replace('d', '')
        try:
            days = int(days_str)
            # Clé unique pour le dictionnaire
            file_key = f"{symbol}_{interval}_{days}d"
            parsed_files_info[file_key] = {'symbol': symbol, 'interval': interval, 'days': days, 'path': f_path}
        except ValueError:
            print(f"Ignoré : nom de fichier non conforme {f_path.name}")
            continue # Ignorer les fichiers qui ne correspondent pas au format attendu

# Options pour les widgets
file_options = {f"{info['symbol']} - {info['interval']} - {info['days']} jours": key 
                for key, info in parsed_files_info.items()}
# Inverser pour avoir le nom lisible comme clé pour le widget, et la clé de fichier comme valeur
# Trier par symbole, puis intervalle, puis jours pour l'affichage dans le dropdown
sorted_file_options_display = sorted(file_options.keys(), key=lambda x: (
    x.split(' - ')[0], # Symbole
    x.split(' - ')[1], # Intervalle
    int(x.split(' - ')[2].split(' ')[0]) # Jours
))


# --- Définition des Widgets ---
file_selector_widget = widgets.Dropdown(
    options=sorted_file_options_display,
    description='Fichier Parquet:',
    style={'description_width': 'initial'},
    layout={'width': 'max-content'} # Pour que le texte entier s'affiche
)

# DatePickers pour la période
start_date_widget = widgets.DatePicker(description='Date de début:', disabled=False)
end_date_widget = widgets.DatePicker(description='Date de fin:', disabled=False)

load_button = widgets.Button(
    description="Charger et Afficher",
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Cliquez pour charger les données et afficher le graphique',
    icon='area-chart' # exemple d'icône font-awesome
)
output_area = widgets.Output()

# --- DataFrame global pour stocker les données chargées ---
# Cela permet aux widgets de date de se mettre à jour quand un nouveau fichier est chargé
current_df = None

# --- Fonctions de Callback ---
def update_date_pickers_range(*args):
    """Met à jour les DatePickers en fonction du fichier sélectionné."""
    global current_df
    with output_area: # Afficher les messages dans la zone de sortie
        selected_file_display_name = file_selector_widget.value
        if not selected_file_display_name:
            start_date_widget.value = None
            end_date_widget.value = None
            current_df = None
            return

        selected_file_key = file_options[selected_file_display_name]
        file_info = parsed_files_info.get(selected_file_key)

        if not file_info:
            print(f"❌ Informations non trouvées pour la sélection : {selected_file_display_name}")
            current_df = None
            return

        file_path = file_info['path']
        
        try:
            df_temp = pd.read_parquet(file_path)
            if 'open_dt' not in df_temp.columns and 'open_time' in df_temp.columns:
                df_temp['open_dt'] = pd.to_datetime(df_temp['open_time'], unit='ms', utc=True)
            elif 'open_dt' not in df_temp.columns:
                 print(f"❌ Le fichier {file_path.name} ne contient ni 'open_dt' ni 'open_time'.")
                 current_df = None
                 return


            if not df_temp.empty and 'open_dt' in df_temp.columns:
                min_date = df_temp['open_dt'].min().date()
                max_date = df_temp['open_dt'].max().date()
                start_date_widget.value = min_date
                end_date_widget.value = max_date
                current_df = df_temp # Stocker le df chargé
                print(f"Fichier {file_path.name} pré-chargé. Période par défaut : {min_date} à {max_date}")
            else:
                start_date_widget.value = None
                end_date_widget.value = None
                current_df = None
                print(f"ℹ️ Le fichier {file_path.name} est vide ou ne contient pas 'open_dt'.")

        except Exception as e:
            print(f"❌ Erreur lors du pré-chargement de {file_path.name} pour les dates : {e}")
            start_date_widget.value = None
            end_date_widget.value = None
            current_df = None


def on_load_button_clicked(b):
    """Fonction appelée lors du clic sur le bouton."""
    global current_df
    with output_area:
        clear_output(wait=True) # Efface la sortie précédente
        print("Traitement en cours...")

        selected_file_display_name = file_selector_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        if not selected_file_display_name:
            print("Veuillez sélectionner un fichier Parquet.")
            return
        
        if current_df is None or current_df.empty:
             # Tentative de recharger si current_df n'est pas setté par update_date_pickers_range
            selected_file_key = file_options[selected_file_display_name]
            file_info = parsed_files_info.get(selected_file_key)
            if file_info:
                try:
                    current_df = pd.read_parquet(file_info['path'])
                    if 'open_dt' not in current_df.columns and 'open_time' in current_df.columns:
                         current_df['open_dt'] = pd.to_datetime(current_df['open_time'], unit='ms', utc=True)
                except Exception as e:
                    print(f"❌ Erreur lors du rechargement du fichier {file_info['path'].name}: {e}")
                    return
            else:
                print(f"❌ Impossible de trouver les informations pour le fichier : {selected_file_display_name}")
                return


        if 'open_dt' not in current_df.columns:
            print("❌ La colonne 'open_dt' est manquante dans le DataFrame chargé.")
            return
            
        # Filtrer par date si les dates sont valides
        df_filtered = current_df.copy()
        if start_date and end_date and start_date <= end_date:
            # Convertir les dates des widgets en datetime pour comparaison (si elles ne le sont pas déjà)
            # Les DatePickers retournent des objets datetime.date, il faut les rendre timezone-aware ou les convertir
            # pour correspondre à 'open_dt' qui est UTC.
            start_dt_utc = datetime.datetime.combine(start_date, datetime.time.min).replace(tzinfo=datetime.timezone.utc)
            end_dt_utc = datetime.datetime.combine(end_date, datetime.time.max).replace(tzinfo=datetime.timezone.utc)

            df_filtered = df_filtered[(df_filtered['open_dt'] >= start_dt_utc) & (df_filtered['open_dt'] <= end_dt_utc)]
            print(f"Filtrage des données entre {start_date} et {end_date}.")
        elif start_date or end_date :
            print("⚠️ Veuillez spécifier une date de début ET une date de fin valides pour filtrer, ou laisser les deux vides pour la période complète.")


        if df_filtered.empty:
            print(f"ℹ️ Aucune donnée à afficher pour la sélection et la période spécifiées.")
            return

        df_plot = df_filtered.set_index('open_dt')

        file_info = parsed_files_info.get(file_options[selected_file_display_name])
        symbol_display = file_info['symbol']
        interval_display = file_info['interval']
        
        print(f"Données pour {symbol_display} ({interval_display}) chargées.")
        print(f"  Nombre de lignes affichées : {len(df_plot):,}")
        if not df_plot.empty:
            print(f"  Période affichée : de {df_plot.index.min().date()} à {df_plot.index.max().date()}")
        print("\n")

        # --- Création du graphique ---
        fig, (ax_price, ax_vol) = plt.subplots(
            2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]}, figsize=(18,9)
        )
        title_suffix = f"De {df_plot.index.min():%Y-%m-%d} à {df_plot.index.max():%Y-%m-%d}" if start_date and end_date else "Période complète"
        fig.suptitle(f"{symbol_display} - Intervalle {interval_display}\n({title_suffix})", fontsize=18, y=0.98)

        ax_price.plot(df_plot.index, df_plot['close'], color='#1f77b4', linewidth=1.5, label="Prix de Clôture")
        ax_price.set_ylabel(f"Prix ({symbol_display.split('USDT')[0] if 'USDT' in symbol_display else symbol_display.split('USDC')[0]}/{'USDT' if 'USDT' in symbol_display else 'USDC'})")
        ax_price.legend(loc="upper left")
        ax_price.grid(True, linestyle='--', alpha=0.7)

        colors = ['#2ca02c' if row['close'] >= row['open'] else '#d62728' for index, row in df_plot.iterrows()]
        # Ajustement de la largeur des barres de volume pour éviter le chevauchement ou qu'elles soient trop fines
        num_points = len(df_plot.index)
        if num_points > 1:
            # Calcule la durée moyenne entre les points pour estimer une largeur de barre
            time_span_days = (df_plot.index[-1] - df_plot.index[0]).total_seconds() / (24 * 3600)
            bar_width_factor = 0.7 # Fraction de l'intervalle moyen entre les points
            if time_span_days > 0 :
                width_in_days = (time_span_days / num_points) * bar_width_factor
            else: # Si très peu de points ou time_span est nul
                 width_in_days = 0.03 # Valeur par défaut
            ax_vol.bar(df_plot.index, df_plot['volume'], color=colors, width=width_in_days, alpha=0.8)
        elif num_points == 1: # Un seul point
             ax_vol.bar(df_plot.index, df_plot['volume'], color=colors, width=0.03, alpha=0.8)


        ax_vol.set_ylabel("Volume")
        ax_vol.grid(True, linestyle='--', alpha=0.7)
        
        ax_vol.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
        plt.setp(ax_vol.get_xticklabels(), rotation=30, ha='right') # Utiliser plt.setp pour plus de robustesse

        fig.tight_layout(rect=[0, 0, 1, 0.94]) # Ajuster pour le suptitle plus grand
        plt.show()

# --- Lier les callbacks aux widgets ---
# Mettre à jour les date pickers quand le fichier sélectionné change
file_selector_widget.observe(update_date_pickers_range, names='value')
# Lier la fonction au clic du bouton
load_button.on_click(on_load_button_clicked)


# --- Affichage des widgets ---
# Panneau de contrôle pour la sélection du fichier et des dates
controls_selection = VBox([
    file_selector_widget,
    HBox([start_date_widget, end_date_widget])
])

# Assemblage final des widgets
ui = VBox([controls_selection, load_button, output_area])
display(ui)

# --- Chargement initial (optionnel, pour la première fois) ---
# Simuler une première mise à jour pour peupler les date pickers si des fichiers existent
if file_options:
    update_date_pickers_range()

Répertoire de données configuré : /Users/sloutmyv/Documents/Github/crypto-ai-bot-v1/data
